In [43]:
# Importamos las bibliotecas necesarias y el dataset previamente limpiado

import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold

from itertools import combinations
import sklearn.metrics as metrics

In [24]:
data_gym = pd.read_excel('DatosParaModelo.xlsx')

In [25]:
# Se eliminan los registros del último mes para aquellos IDs que sólo tienen un registro

# Se obtienen los IDs que sólo tienen 1 registro y los IDs presentes en el último mes
Conteo_registros = data_gym['IdPersona'].value_counts()
ID_multiples_registros = Conteo_registros[Conteo_registros == 1].index
ID_ultimo_mes = data_gym[data_gym['Mes'] == '2023-12-01']['IdPersona'].tolist()

# Se genera una lista con los IDs comunes en ambas listas y se descartan del DataFrame
IDs_eliminar = set(ID_multiples_registros).intersection(ID_ultimo_mes)

data_gym_sin_ultimos_registros = data_gym[~data_gym['IdPersona'].isin(IDs_eliminar)]

# Logistic Regression

In [46]:
def Logistic_Regression (DataFrame, Splits):

    lista_AUC_kfolds = []

    Modelo_reg_log = LogisticRegression(penalty=None, max_iter=3000)

    X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
    y = DataFrame['Baja']
    Id_Clientes = DataFrame['IdPersona']
    kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)

    for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

        Modelo_reg_log.fit(X_train, y_train)
        Predicciones_reg = Modelo_reg_log.predict_proba(X_test)
        precision, recall, _ = precision_recall_curve(y_test, Predicciones_reg[:,1])
        AUC_PR = auc(recall, precision)
        # AUC_PR = average_precision_score(y_test, Predicciones_reg[:, 1])
        lista_AUC_kfolds.append(AUC_PR)

    Mean_AUC = np.mean(lista_AUC_kfolds)
    print(f'El AUC del Modelo de Regresión Logística es: {Mean_AUC}')

# Random Forest

In [49]:
def Random_Forest(Profundidades, Estimadores, DataFrame, Splits):

  lista_parametros = []
  lista_AUC = []

  for Depth in Profundidades:
    for n_estimators in Estimadores:
      lista_AUC_kfolds = []

      X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
      y = DataFrame['Baja']
      Id_Clientes = DataFrame['IdPersona']
      kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)
          
      for Train_data, Test_data in kfold.split(X, y, Id_Clientes):
        X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
        y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]    
    
        Modelo_random_forest = RandomForestClassifier(max_depth=Depth, n_estimators=n_estimators, random_state=1).fit(X_train, y_train)
        Predicciones_rndm_frst = Modelo_random_forest.predict_proba(X_test)
        precision, recall, _ = precision_recall_curve(y_test, Predicciones_rndm_frst[:,1])
        AUC_PR = auc(recall, precision)
        # AUC_PR = average_precision_score(y_test, Predicciones_rndm_frst[:, 1])
        lista_AUC_kfolds.append(AUC_PR)

      Mean_AUC = np.mean(lista_AUC_kfolds)
      lista_AUC.append(Mean_AUC)    
      lista_parametros.append([Depth, n_estimators])

  AUC_max_value = np.argmax(lista_AUC)
  print(f'El AUC del modelo Random Forest es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Gradient Boosting Classifier

In [50]:
def Gradient_Boosting (Profundidades, Estimadores, DataFrame, Splits):

    lista_parametros = []
    lista_AUC = []

    for Depth in Profundidades:
        for n_estimators in Estimadores:
            lista_AUC_kfolds = []
        
            X = DataFrame.drop(columns = ['IdPersona', 'Baja', 'Mes'])
            y = DataFrame['Baja']
            Id_Clientes = DataFrame['IdPersona']
            kfold = StratifiedKFold(n_splits=Splits, shuffle=True, random_state=1)
            
            for Train_data, Test_data in kfold.split(X, y, groups = Id_Clientes):
                X_train, X_test = X.iloc[Train_data], X.iloc[Test_data]
                y_train, y_test = y.iloc[Train_data], y.iloc[Test_data]

                Modelo_xgboost = xgb.XGBClassifier(n_estimators=n_estimators, max_depth=Depth, objective='binary:logistic').fit(X_train, y_train)
                Predicciones_xgb = Modelo_xgboost.predict_proba(X_test)
                precision, recall, _ = precision_recall_curve(y_test, Predicciones_xgb[:,1])
                AUC_PR = auc(recall, precision)
                # AUC_PR = average_precision_score(y_test, Predicciones_xgb[:, 1])
                lista_AUC_kfolds.append(AUC_PR)

            Mean_AUC = np.mean(lista_AUC_kfolds)
            lista_AUC.append(Mean_AUC)
            lista_parametros.append([Depth, n_estimators])

    AUC_max_value = np.argmax(lista_AUC)
    print(f'El AUC del modelo XGBoost es {lista_AUC[AUC_max_value]}, para un nivel de profundidad de {lista_parametros[AUC_max_value][0]} y consta de {lista_parametros[AUC_max_value][1]} árboles de decisión')

# Edición DataFrame - Descarte de Columnas

In [29]:
# Grupos de columnas que queremos ignorar en el Dataset para comprobar si aportan valor al modelo
Columna_TipoAbono = ['TipoAbono_- FREE FAMILIAR 2 ADULTS', 'TipoAbono_- QUOTA FREE','TipoAbono_. QUOTA CLUB', 'TipoAbono_ABONAMENT GENERAL', 'TipoAbono_ADULT SENSE TR AMB 33% DISC.', 'TipoAbono_BAIXA TEMPORAL', 'TipoAbono_ESPORTSALUS', 'TipoAbono_FAMILIAR 2 ADULTS', 'TipoAbono_FAMILIAR 3 ADULTS', 'TipoAbono_FAMILIAR MONOPARENTAL']
Columna_MesAlta = ['MesAlta_1', 'MesAlta_2', 'MesAlta_3', 'MesAlta_4', 'MesAlta_5', 'MesAlta_6', 'MesAlta_7', 'MesAlta_8', 'MesAlta_9', 'MesAlta_10', 'MesAlta_11', 'MesAlta_12']
Columna_MesActual = ['MesActual_1', 'MesActual_2', 'MesActual_3', 'MesActual_4', 'MesActual_5', 'MesActual_6', 'MesActual_7', 'MesActual_8', 'MesActual_9', 'MesActual_10', 'MesActual_11', 'MesActual_12']

Grupos_columnas = [Columna_TipoAbono, Columna_MesAlta, Columna_MesActual]
Combinaciones_grupos = []

# Agregamos a la lista Combinaciones_grupos las diferentes combinatorias entre los 3 grupos 
for grupo in range(len(Grupos_columnas) + 1):
    Combinaciones_grupos.extend(combinations(Grupos_columnas, grupo))

# Iteramos sobre la lista de combinaciones para excluir las columnas al Dataset y crear una nueva lista con los diferentes DataFrames
Lista_DataFrames = []
for Combinacion in Combinaciones_grupos:
    
    Columnas_prescindir = [columna for groupo in Combinacion for columna in groupo]
    DF_modificado = data_gym.drop(columns = Columnas_prescindir)
    Lista_DataFrames.append(DF_modificado)

# Variables de los Modelos

In [30]:
Splits = 50
Profundidades = [2, 3, 4, 5, 6, 7, 8]
Estimadores = [20, 40, 60, 80, 100]

# Ejecución de Modelos

In [41]:
pd.set_option('display.max_rows', 2)

In [47]:
for Numero_DF, DataFrame in enumerate(Lista_DataFrames):

    print(f"Evaluando DF {Numero_DF + 1}/{len(Lista_DataFrames)}")
    Logistic_Regression(DataFrame, Splits)
    

Evaluando DF 1/8
El AUC del Modelo de Regresión Logística es: 0.19233915546522262
Evaluando DF 2/8
El AUC del Modelo de Regresión Logística es: 0.1869432709545537
Evaluando DF 3/8
El AUC del Modelo de Regresión Logística es: 0.19149161512717341
Evaluando DF 4/8
El AUC del Modelo de Regresión Logística es: 0.18828937013868852
Evaluando DF 5/8
El AUC del Modelo de Regresión Logística es: 0.18702461461652867
Evaluando DF 6/8
El AUC del Modelo de Regresión Logística es: 0.184452752773014
Evaluando DF 7/8
El AUC del Modelo de Regresión Logística es: 0.18827196217904782
Evaluando DF 8/8
El AUC del Modelo de Regresión Logística es: 0.1847495067792358


In [48]:
for Numero_DF, DataFrame in enumerate(Lista_DataFrames):

    print(f"Evaluando DF {Numero_DF + 1}/{len(Lista_DataFrames)}")
    Gradient_Boosting(Profundidades, Estimadores, DataFrame, Splits)

Evaluando DF 1/8
El AUC del modelo XGBoost es 0.24520808947819128, para un nivel de profundidad de 4 y consta de 60 árboles de decisión
Evaluando DF 2/8
El AUC del modelo XGBoost es 0.23629786781209153, para un nivel de profundidad de 3 y consta de 100 árboles de decisión
Evaluando DF 3/8
El AUC del modelo XGBoost es 0.24439269873360664, para un nivel de profundidad de 5 y consta de 40 árboles de decisión
Evaluando DF 4/8
El AUC del modelo XGBoost es 0.2384299154455023, para un nivel de profundidad de 6 y consta de 20 árboles de decisión
Evaluando DF 5/8
El AUC del modelo XGBoost es 0.2382545867978439, para un nivel de profundidad de 3 y consta de 80 árboles de decisión
Evaluando DF 6/8
El AUC del modelo XGBoost es 0.2329981221441878, para un nivel de profundidad de 6 y consta de 20 árboles de decisión
Evaluando DF 7/8
El AUC del modelo XGBoost es 0.23800399602052835, para un nivel de profundidad de 6 y consta de 20 árboles de decisión
Evaluando DF 8/8
El AUC del modelo XGBoost es 0.22

In [51]:
for Numero_DF, DataFrame in enumerate(Lista_DataFrames):

    print(f"Evaluando DF {Numero_DF + 1}/{len(Lista_DataFrames)}")
    Random_Forest(Profundidades, Estimadores, DataFrame, Splits)

Evaluando DF 1/8
El AUC del modelo Random Forest es 0.23537458222401117, para un nivel de profundidad de 8 y consta de 100 árboles de decisión
Evaluando DF 2/8
El AUC del modelo Random Forest es 0.22902882896584079, para un nivel de profundidad de 8 y consta de 80 árboles de decisión
Evaluando DF 3/8
El AUC del modelo Random Forest es 0.23368034035827556, para un nivel de profundidad de 8 y consta de 60 árboles de decisión
Evaluando DF 4/8
El AUC del modelo Random Forest es 0.22917162055431997, para un nivel de profundidad de 8 y consta de 100 árboles de decisión
Evaluando DF 5/8
El AUC del modelo Random Forest es 0.22923027822862674, para un nivel de profundidad de 8 y consta de 80 árboles de decisión
Evaluando DF 6/8
El AUC del modelo Random Forest es 0.2242532332921769, para un nivel de profundidad de 8 y consta de 100 árboles de decisión
Evaluando DF 7/8
El AUC del modelo Random Forest es 0.23085984010705224, para un nivel de profundidad de 8 y consta de 100 árboles de decisión
Eva

In [52]:
for Numero_DF, DataFrame in enumerate(Lista_DataFrames):
    print(f"Evaluando DF {Numero_DF + 1}/{len(Lista_DataFrames)}")
    print(DataFrame.columns)
    print()

Evaluando DF 1/8
Index(['IdPersona', 'Mes', 'Edad', 'Genero_masculino', 'EsAbonado', 'Accesos',
       'Accesos-1', 'Accesos-2', 'AccesosTrimestrales', 'MesesAntiguedad',
       'Baja', 'TipoAbono_- FREE FAMILIAR 2 ADULTS', 'TipoAbono_- QUOTA FREE',
       'TipoAbono_. QUOTA CLUB', 'TipoAbono_ABONAMENT GENERAL',
       'TipoAbono_ADULT SENSE TR AMB 33% DISC.', 'TipoAbono_BAIXA TEMPORAL',
       'TipoAbono_ESPORTSALUS', 'TipoAbono_FAMILIAR 2 ADULTS',
       'TipoAbono_FAMILIAR 3 ADULTS', 'TipoAbono_FAMILIAR MONOPARENTAL',
       'MesAlta_1', 'MesAlta_2', 'MesAlta_3', 'MesAlta_4', 'MesAlta_5',
       'MesAlta_6', 'MesAlta_7', 'MesAlta_8', 'MesAlta_9', 'MesAlta_10',
       'MesAlta_11', 'MesAlta_12', 'MesActual_1', 'MesActual_2', 'MesActual_3',
       'MesActual_4', 'MesActual_5', 'MesActual_6', 'MesActual_7',
       'MesActual_8', 'MesActual_9', 'MesActual_10', 'MesActual_11',
       'MesActual_12'],
      dtype='object')

Evaluando DF 2/8
Index(['IdPersona', 'Mes', 'Edad', 'Genero_mascu